]言語モデルのファインチューニング　2025/006/19


*   最後のcell[]まで実行済み



# 第13章 RAG

## 13.3 RAG 向けに LLM を指示チューニングする

### 13.3.2 指示チューニングしたモデルを LangChain で使う

#### 環境の準備

In [1]:
!pip install transformers[torch,sentencepiece] langchain langchain-community langchain-huggingface faiss-cpu jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.3/754.3 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
from huggingface_hub import notebook_login

# Hugging Face Hubにログイン
notebook_login()

In [3]:
from transformers.trainer_utils import set_seed

# 乱数のシードを設定
set_seed(42)

#### Chat Model の作成

In [4]:
import torch
from langchain_huggingface import (
    ChatHuggingFace,
    HuggingFacePipeline,
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
)

# Hugging Face Hubにおけるモデル名を指定
model_name = "llm-book/Swallow-7b-hf-oasst1-21k-ja-aio-retriever"

# モデルを読み込む
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# トークナイザを読み込む
tokenizer = AutoTokenizer.from_pretrained(model_name)

# テキスト生成用のパラメータを指定
generation_config = {
    "max_new_tokens": 32,
    "do_sample": False,
    "temperature": None,
    "top_p": None,
}

# テキスト生成を行うパイプラインを作成
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    **generation_config,
)

# パイプラインからLangChainのLLMコンポーネントを作成
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# LLMコンポーネントを元にChat Modelコンポーネントを作成
chat_model = ChatHuggingFace(llm=llm, tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/770 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.77G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.30M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

Device set to use cuda:0


#### Embedding Model の作成

In [5]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Hugging Face Hubにおけるモデル名を指定
embedding_model_name = "BAAI/bge-m3"

# モデル名からEmbedding Modelを初期化
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"model_kwargs": {"torch_dtype": torch.float16}},
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

#### データストアの構築

In [6]:
# 検索対象の文書集合のファイルをダウンロード
!wget  \
https://github.com/ghmagazine/llm-book/raw/main/chapter13/docs.json

--2025-06-19 08:13:57--  https://github.com/ghmagazine/llm-book/raw/main/chapter13/docs.json
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ghmagazine/llm-book/main/chapter13/docs.json [following]
--2025-06-19 08:13:58--  https://raw.githubusercontent.com/ghmagazine/llm-book/main/chapter13/docs.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1178382 (1.1M) [text/plain]
Saving to: ‘docs.json’

docs.json           100%[===================>]   1.12M  --.-KB/s    in 0.009s  

2025-06-19 08:13:58 (128 MB/s) - ‘docs.json’ saved [1178382/1178382]



In [7]:
from langchain_community.document_loaders import JSONLoader

# JSONファイルから文書を読み込むためのDocument Loaderを初期化
document_loader = JSONLoader(
    file_path="./docs.json",  # 読み込みを行うファイル
    jq_schema=".text",  # 読み込み対象のフィールド
    json_lines=True,  # JSON Lines形式のファイルであることを指定
)

# 文書の読み込みを実行
documents = document_loader.load()

# 読み込まれた文書数を確認
print(len(documents))

103


In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 文書を指定した文字数で分割するText Splitterを初期化
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,  # 分割する最大文字数
    chunk_overlap=100,  # 分割された文書間で重複させる最大文字数
    add_start_index=True,  # 元の文書における開始位置の情報を付与
)

# 文書の分割を実行
split_documents = text_splitter.split_documents(documents)

# 分割後の文書数を確認
print(len(split_documents))

1475


#### 検索対象の文書のベクトルインデックスの作成

In [9]:
from langchain_community.vectorstores import FAISS

# 分割後の文書と文埋め込みモデルを用いて、Faissのベクトルインデックスを作成
vectorstore = FAISS.from_documents(split_documents, embedding_model)

# ベクトルインデックスに登録された文書数を確認
print(vectorstore.index.ntotal)

1475


#### Retriever コンポーネントの作成

In [10]:
# ベクトルインデックスを元に文書の検索を行うRetrieverを初期化
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [11]:
from pprint import pprint

# 文書の検索を実行
retrieved_documents = retriever.invoke("四国地方で一番高い山は？")

# 検索された文書を確認
pprint(retrieved_documents)

[Document(id='7ed1b81a-f361-4978-bbbc-b9a51440819c', metadata={'source': '/content/docs.json', 'seq_num': 26, 'start_index': 0}, page_content='この項目に含まれる文字「鎚」は、オペレーティングシステムやブラウザなどの環境により表示が異なります。 石鎚山（いしづちさん、いしづちやま）は、四国山地西部に位置する標高1,982 mの山で、近畿以西を「西日本」とした場合の西日本最高峰で、山頂から望む展望が四国八十八景64番に選定。愛媛県西条市と久万高原町の境界に位置する。 石鉄山、石鈇山、石土山、石槌山とも表記され、伊予の高嶺とも呼ばれる。『日本霊異記』には「石槌山」と記され、延喜式の神名帳（延喜式神名帳）では「石鉄神社」と記されている。前神寺および横峰寺では「石鈇山（しゃくまざん）」とも呼ぶ。'),
 Document(id='2fb1a497-4c68-4cf8-9d66-bb826538605a', metadata={'source': '/content/docs.json', 'seq_num': 1, 'start_index': 0}, page_content='富士山（ふじさん）は、静岡県（富士宮市、富士市、裾野市、御殿場市、駿東郡小山町）と山梨県（富士吉田市、南都留郡鳴沢村）に跨る活火山である。標高3776.12 m、日本最高峰（剣ヶ峰）の独立峰で、その優美な風貌は日本国外でも日本の象徴として広く知られている。 数多くの芸術作品の題材とされ芸術面のみならず、気候や地層など地質学的にも社会に大きな影響を与えている。懸垂曲線の山容を有した玄武岩質成層火山で構成され、その山体は駿河湾の海岸まで及ぶ。'),
 Document(id='f0669636-05b0-48b6-ab93-9310ba22f5d3', metadata={'source': '/content/docs.json', 'seq_num': 96, 'start_index': 0}, page_content='四阿山（あずまやさん）は、長野県と群馬県の県境に跨る山。標高2,354 m。日本百名山の一つに数えられている。吾妻山・吾嬬山（あがつま

In [18]:
from pprint import pprint

# 文書の検索を実行
retrieved_documents = retriever.invoke("現在の核融合炉の開発状況は？")

# 検索された文書を確認
pprint(retrieved_documents)

[Document(id='f670e6ab-c930-4601-8334-e92e3e4d9d24', metadata={'source': '/content/docs.json', 'seq_num': 16, 'start_index': 4393}, page_content='火砕流（ブロックアンドアッシュフロー）堆積物である。大山火山で最新の活動である三鈷峰溶岩ドームと阿弥陀川火砕流堆積物（約2万800年前）は、大山北東部に噴出したデイサイト質溶岩ドームと、それの一部崩壊によって阿弥陀川沿いに流下した火砕流（ブロックアンドアッシュフロー）堆積物である。なお、大山南麓の江府町御机で約3000年前の火砕流堆積物とその降下火砕物を発見したとする報告もある。しかし、現在のところ大山火山は活火山（概ね過去1万年以内に噴火した火山及び現在活発な噴気活動のある火山）に指定されていない。'),
 Document(id='2e14ffff-8497-4c39-888a-f1f6461545af', metadata={'source': '/content/docs.json', 'seq_num': 36, 'start_index': 6490}, page_content='過去の噴火事例から避難経路などを取りまとめたハザードマップの作成が行われている。また、長野県小諸市の千曲川河畔まで溶岩流が流れた痕跡や群馬県側の吾妻川では、山体崩壊に伴い大規模な土石流が流下し、前橋市付近までの広い地域に土砂が堆積した形跡があり、山体付近だけの問題ではない。 火山噴火予知連絡会によって火山防災のために監視・観測体制の充実等の必要がある火山に選定されている。また、東京大学地震研究所等により365日24時間の観測が行われている。 前掛山 - 中央火口丘、釜山火口（現在の火口）がある。 黒斑山 - 古期成層火山のカルデラ縁。 側火山 小浅間山 - 溶岩ドーム。釜山火口から東3.8km。 離山 - 溶岩ドーム、比高は約200m、釜山火口から南東10km。 石尊山 - 前掛山の南側中腹に位置する溶岩ドーム、比高約250m、釜山火口から南西3.3km。'),
 Document(id='946d0eca-90ae-4bdd-814e-5765b0326fc0', metadata={'

#### RAG の Chain の構築と実行

In [12]:
from langchain_core.prompts import ChatPromptTemplate

# 任意のqueryからメッセージを構築するPrompt Templateを作成
rag_prompt_text = (
    "あなたには今からクイズに答えてもらいます。"
    "問題を与えますので、その解答のみを簡潔に出力してください。\n"
    "また解答の参考になりうるテキストを与えます。"
    "解答を含まない場合もあるのでその場合は無視してください。\n\n"
    "---\n{context}\n---\n\n問題: {query}"
)
rag_prompt_template = ChatPromptTemplate.from_messages(
    [("user", rag_prompt_text)]
)

In [13]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

def format_documents_func(documents: list[Document]) -> str:
    """文書のリストを改行で連結した一つの文字列として返す"""
    return "\n\n".join(
        document.page_content for document in documents
    )

# 定義した関数の処理を行うRunnableを作成
format_documents = RunnableLambda(format_documents_func)

In [14]:
from langchain_core.prompt_values import ChatPromptValue

def chat_model_resp_only_func(
    chat_prompt_value: ChatPromptValue,
) -> str:
    """chat_modelにchat_prompt_valueを入力し、
    出力からモデルの応答部分のみを文字列で返す"""
    chat_prompt = chat_model._to_chat_prompt(
        chat_prompt_value.messages
    )
    chat_output_message = chat_model.invoke(chat_prompt_value)
    response_text = chat_output_message.content[len(chat_prompt) :]
    return response_text

# 定義した関数の処理を行うRunnableを作成
chat_model_resp_only = RunnableLambda(chat_model_resp_only_func)

In [15]:
from langchain_core.runnables import RunnablePassthrough

# RAGの一連の処理を行うChainを作成
rag_chain = (
    {
        "context": retriever | format_documents,
        "query": RunnablePassthrough(),
    }
    | rag_prompt_template
    | chat_model_resp_only
)

In [16]:
# Chainを実行し、結果を確認
rag_chain_output = rag_chain.invoke("四国地方で一番高い山は？")
print(rag_chain_output)

石鎚山


In [17]:
# Chainを実行し、結果を確認
rag_chain_output = rag_chain.invoke("現在の核融合炉の開発状況は？")
print(rag_chain_output)

核融合炉
